In [1]:
import os
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import mammoth
import json
from urllib.parse import urlparse, urljoin, unquote
import re
import PyPDF2
import pdfplumber
import docx
import pypandoc

from tld import get_fld

In [2]:
import pypandoc
#pypandoc.download_pandoc()

In [3]:
# Create a folder for downloads
DOWNLOAD_DIR = "999_downloaded_documents_link1"
os.makedirs(DOWNLOAD_DIR, exist_ok=True)

In [4]:
# Read URLs from file
def read_urls(file_name='999.web_urls.txt'):
    with open(file_name, 'r') as f:
        return [line.strip() for line in f.readlines()]

In [5]:
# ---> Función auxiliar para verificar si un enlace es interno (pertenece al mismo dominio)
# Esta funcion es un componente del proceso en bloque 7 que "scrape and store document links in JSON with parent page path segments" ---> esta funcion evita enviar request a links por fuera del dominio al que estamos scrapping


# Helper function to check if a link is internal (within the same domain)
def is_internal_link(url, base_url):
    # Parse both URLs
    parsed_url = urlparse(url)
    parsed_base_url = urlparse(base_url)

    # Check if the domain of the URL matches the base URL's domain
    # TRUE si el dominio es el mismo que el base pero FULL URL, RELATIVE PATH, SAME DOMAIN BUT DIFFERENT PROTOCOL else FALSE
    return parsed_url.netloc == parsed_base_url.netloc or parsed_url.netloc == ''

In [6]:
# Verifica si el contenido de una respuesta es un formato binario (como PDF o Word)
#'application/pdf' for PDFs,'application/msword' for Word documents, and 'application/vnd' for various document formats (e.g., Excel or OpenDocument).
# THIS FUNCTION IS A COMPONENT FOR get_document_links() Function

def is_binary_content(response):
    content_type = response.headers.get('Content-Type', '').lower()
    return 'application/pdf' in content_type or 'application/msword' in content_type or 'application/vnd' in content_type

In [7]:
# Función para limpiar nombres de archivos extraídos de URLs
# The cleaned file_name is returned, making it suitable for use as a filename.
# THIS FUNCTION IS A COMPONENT FOR download_file() Function

# Function to clean filenames
def clean_filename(href):
    # Extract the file name from the URL
    file_name = unquote(os.path.basename(urlparse(href).path))
    # Remove invalid characters for filenames using a regex
    file_name = re.sub(r'[<>:"/\\|?*]', '_', file_name)
    # Fallback if the URL doesn't have a file name
    if not file_name:
        file_name = "downloaded_file"
    return file_name

In [8]:
# Función para obtener enlaces de documentos desde un sitio web, navegando recursivamente por enlaces internos.
# THIS FUNCTION IS A COMPONEN FOR scrape_and_store_links() and scrape_documents_from_website() FUNCTIONS

# Recursively fetch document links from a website, traversing internal links.
def get_document_links(url, base_url, visited=None):
    if visited is None:
        visited = set()

    doc_links = []

    # Ensure we do not revisit the same page more than once
    if url in visited:
        return doc_links
    visited.add(url)

    try:
        sys.stdout.write(f"Processing: {url}\r")
        sys.stdout.flush()

        response = requests.get(url)

        # Skip binary files (like PDFs, DOCs) to avoid trying to parse them
        if is_binary_content(response):
            sys.stdout.write(f"Skipping binary file: {url}\r")
            sys.stdout.flush()
            return doc_links

        # Parse HTML content with BeautifulSoup
        soup = BeautifulSoup(response.text, 'html.parser')

        # Find all document links on the page (pdf, doc, docx)
        for link in soup.find_all('a', href=True):
            href = link['href']
            if href.startswith(('tel:', 'mailto:')):
                continue
            if href.endswith(('pdf', 'doc', 'docx','doc')):
                full_url = href if href.startswith('http') else urljoin(url, href)
                # Append both the parent page (url) and the document link (full_url)
                doc_links.append([url, (link, full_url)])  # Return parent page and document link as 2D array

        # Now, find all internal links to recursively navigate
        for link in soup.find_all('a', href=True):
            href = link['href']
            full_url = href if href.startswith('http') else urljoin(url, href)
            if is_internal_link(full_url, base_url) and full_url not in visited:
                doc_links.extend(get_document_links(full_url, base_url, visited))

    except Exception as e:
        print(f"\nError fetching {url}: {e}")

    return doc_links


Purpose
The function get_document_links() returns a list of document links found on the site, with each link paired with its originating page. This approach is useful for collecting downloadable files from an entire website, starting from a base URL and exploring all reachable internal pages.

OUTCOME EXAMPLE: 

doc_links = 
[
    [
        'https://example.com/page1', 
        ('Document 1', 'https://example.com/files/doc1.pdf')
    ],
    [
        'https://example.com/page1', 
        ('Document 2', 'https://example.com/files/doc2.doc')
    ],
    [
        'https://example.com/page2', 
        ('Research Paper', 'https://example.com/files/research_paper.pdf')
    ],
    [
        'https://example.com/page3', 
        ('Report 2023', 'https://example.com/files/report.docx')
    ]
]


In [9]:
# Función para convertir una cadena de texto a formato camelCase
# THIS FUNCTION IS A COMPONENT OF scrape_and_store_links() function
# camelCase, un estilo común de escritura en programación, especialmente útil para nombres de variables y funciones.
# La función convierte texto a camelCase (ej. nombreCompleto), ideal para nombres de variables y claves en JSON, siguiendo convenciones de programación

def to_camel_case(text):
    # Return empty string if the input is empty or None
    if not text:
        return ''
    text = re.sub(r'[_\-]+', ' ', text)
    words = text.split()
    if not words:
        return ''
    camel_case_text = words[0].lower() + ''.join(word.capitalize() for word in words[1:])
    
    return camel_case_text

In [10]:
# Función para extraer y almacenar enlaces de documentos en JSON, incluyendo los segmentos de ruta de la página principal
# Esta funcion crea el archivo .json "scraped_links_y-m-d-H-M-S) 
# Extrae metadata parent_page, tittle, href y file_url
# Esta funcion se activa y es componente del ultimo bloque de codigo main() function

def scrape_and_store_links(urls):
    data = []  # List to hold the scraped links
    timestamp =datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
    
    ##timestamp = datetime.now().isoformat()  # Timestamp for saving the file

    for url in urls:
        print(f"Processing URL: {url}")
        base_url = url  # This is the parent page (where the link is found)

        doc_links = get_document_links(base_url, base_url, visited=set())  # Scrape the links from the base URL

        for parent_page, (link, doc_url) in doc_links:  # Extract parent_page and document link from the 2D array
            # Get the title from the link element (use link text)
            title = link.string or link.get_text().strip() or "No title"  # Fallback to "No title" if empty

            # Extract the path segments from the parent page (base_url)
            parsed_url = urlparse(parent_page)  # Use the parent_page returned by get_document_links
            path_segments = parsed_url.path.strip('/').split('/')
            
            # If path is empty, use domain (netloc) as the folder name
            if not path_segments or path_segments == ['']:
                path_segments = [parsed_url.netloc]  # Fallback to the domain if the path is empty
            else:
                # Convert to camelCase and skip empty segments
                path_segments = [to_camel_case(segment) for segment in path_segments if segment]

            # Append the details of each link to the data list, including the parent page path segments
            data.append({
                "timestamp": timestamp,
                "parent_page": path_segments,  # Store the path segments as an array
                "title": title,
                "href": link['href'],  # Extract href directly from the <a> tag (the document link)
                "file_url": doc_url,  # Full URL of the document
            })

    # Convert the list of links into a DataFrame and save as JSON
    df = pd.DataFrame(data)
    output_file = f"scraped_links_{timestamp}.json"
    df.to_json(output_file, orient="records", indent=4)
    print(f"Scraping complete. Links saved to {output_file}")
    return output_file


La función scrape_and_store_links extrae la siguiente metadata para cada documento encontrado:

timestamp: La fecha y hora exacta de la extracción, en formato YYYY-MM-DD_HH-MM-SS. Esto facilita saber cuándo se realizó el scraping.

parent_page: Los segmentos de la ruta de la página principal donde se encontró el enlace. Cada segmento de la URL se guarda como un elemento en una lista, usando formato camelCase (ej. ["seccionPrincipal", "subseccion"]). Si no hay ruta, se usa el dominio.

title: El texto del enlace al documento, que generalmente describe el documento o su título. Si está vacío, se establece como "No title".

href: La URL parcial extraída directamente del atributo href del enlace (<a>) en la página.

file_url: La URL completa del documento (PDF, DOC, etc.), que se puede utilizar para descargar el archivo.

Este conjunto de metadata organiza los documentos y su procedencia, siendo útil para estructurar y consultar enlaces descargables.

In [11]:
# Main function to initiate scraping for documents
# Función principal para iniciar la extracción de documentos desde un sitio web
# Esta funcion se activa y es componente del ultimo bloque de codigo main() function



def scrape_documents_from_website(url):
    base_url = url  # The starting URL
    visited = set()  # To keep track of visited URLs
    doc_links = get_document_links(base_url, base_url, visited)

    print(f"Found {len(doc_links)} document links:")
    for title, link in doc_links:
        print(f"- {title}: {link}")

    return doc_links

LOGICA DE scrape_documents_from_website(x):

Si llamamos a la función con un sitio web como https://example.com, el output en la consola podría verse así:
scrape_documents_from_website("https://example.com")

OUTPUT IS A LIST OF TUPPLES
Each tuple in the list contains two elements: the title of the document and the document’s URL.

OUTPUT:
Found 3 document links:
- Document 1: https://example.com/files/doc1.pdf
- Report 2022: https://example.com/files/report2022.doc
- Guide: https://example.com/downloads/guide.docx

Explicación
La función muestra:

El total de enlaces de documentos encontrados (en este caso, 3).
Una lista donde cada línea representa un documento, mostrando su título (Document 1, Report 2022, Guide) y el enlace directo al archivo (doc1.pdf, report2022.doc, guide.docx).
Este formato hace fácil ver qué documentos fueron encontrados y sus URLs exactas, útiles para confirmar que los enlaces fueron correctamente detectados.

In [12]:
# Function to create subdirectories based on the parent page path segments
# Función para crear subdirectorios basados en los segmentos de ruta de la página principal
# This function organizes documents into a structured folder system based on the webpage hierarchy, making it easier to manage downloaded files according to their origin.
# This function is called and a component for download_file() function


def create_directory_structure(parent_page_segments, doc_url):
    # Convert the path segments array into a directory path
    directory_path = os.path.join(DOWNLOAD_DIR, *parent_page_segments)

    # Create the directories if they don't exist
    os.makedirs(directory_path, exist_ok=True)

    return directory_path

In [13]:
# Function to download a document and save it in the appropriate subdirectory
# Función para descargar un documento y guardarlo en el subdirectorio correspondiente
# Esta funcion se activa y es componente del ultimo bloque de codigo main() function

def download_file(base_url, doc_url, context_href):
    # Use the href to generate a meaningful name
    file_name = clean_filename(context_href)

    # Create the directory structure based on Page and Section
    directory_path = create_directory_structure(base_url, doc_url)

    # Create the full file path to save the file
    file_path = os.path.join(directory_path, file_name)

    try:
        response = requests.get(doc_url)
        with open(file_path, 'wb') as f:
            f.write(response.content)
        print(f"\nDownloaded: {file_name} to {directory_path}")
        return file_path
    except Exception as e:
        print(f"\nFailed to download {doc_url}: {e}")
        return None

In [14]:
import logging

# Set up logging configuration
logging.basicConfig(
    filename="scraping_log.txt",
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

logging.info("Starting the scraping process...")

def download_file(base_url, doc_url, context_href):
    # Use the href to generate a meaningful name
    file_name = clean_filename(context_href)
    directory_path = create_directory_structure(base_url, doc_url)
    file_path = os.path.join(directory_path, file_name)

    try:
        response = requests.get(doc_url, timeout=10)  # Add timeout here
        with open(file_path, 'wb') as f:
            f.write(response.content)
        logging.info(f"Downloaded: {file_name} to {directory_path}")
        return file_path
    except requests.exceptions.Timeout:
        logging.warning(f"Timeout occurred for {doc_url}")
        return None
    except Exception as e:
        logging.error(f"Failed to download {doc_url}: {e}")
        return None


In [15]:

import win32com.client as win32
from docx import Document  # For extracting text from .docx
import mammoth
import pdfplumber
import os
import logging
import re

# Setup logging
logging.basicConfig(level=logging.INFO)

# Extract text from file based on its extension
def extract_text_from_file(file_path):
    if file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    elif file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".doc"):
        return extract_text_from_doc(file_path)
    else:
        logging.error(f"Unsupported file format for {file_path}")
        return None

# Extract text from .docx files using mammoth and clean Unicode escapes
def extract_text_from_docx(file_path):
    try:
        with open(file_path, "rb") as docx_file:
            result = mammoth.extract_raw_text(docx_file)
            text = result.value
            return text.encode().decode('unicode_escape') if text else "No text found in the DOCX file"
    except Exception as e:
        logging.error(f"Error extracting text from .docx file {file_path}: {e}")
        return None

# Extract text from .pdf files using pdfplumber and clean Unicode escapes
def extract_text_from_pdf(file_path):
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text() or ""
                text += page_text  # Concatenate raw text
            return text.encode().decode('unicode_escape') if text else "No text found in the PDF file"
    except Exception as e:
        logging.error(f"Error extracting text from PDF file {file_path}: {e}")
        return None

# Extract text from .doc files using win32com
def extract_text_from_doc(file_path):
    try:
        word = win32.Dispatch("Word.Application")
        word.Visible = False
        doc = word.Documents.Open(file_path)
        text = doc.Content.Text
        doc.Close()
        word.Quit()
        return text.encode().decode("unicode_escape", "replace")
    except Exception as e:
        logging.error(f"Error extracting text from .doc file {file_path}: {e}")
        return None

# Function to replace characters in JSON files
def replace_characters_in_json_file(output_file):
    replacements = {
        "\u00e1": "á", "\u00e9": "é", "\u00ed": "í", "\u00f3": "ó", "\u00fa": "ú",
        "\u00c1": "Á", "\u00c9": "É", "\u00cd": "Í", "\u00d3": "Ó", "\u00da": "Ú",
        "\u00f1": "ñ", "\u00d1": "Ñ"
    }

    try:
        with open(output_file, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
        
        json_str = json.dumps(json_data)
        
        for unicode_seq, char in replacements.items():
            json_str = re.sub(unicode_seq, char, json_str)

        updated_json_data = json.loads(json_str)

        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(updated_json_data, file, ensure_ascii=False, indent=4)
        
        print("Character replacements in JSON file completed successfully.")
    
    except Exception as e:
        print(f"An error occurred while processing the JSON file: {e}")

# Example usage
file_path = "your_file_path_here"
extracted_text = extract_text_from_file(file_path)
print(extracted_text)


None


In [16]:

import win32com.client as win32
from docx import Document  # For extracting text from .docx
import mammoth
import pdfplumber
import os
import logging
import re

# Setup logging
logging.basicConfig(level=logging.INFO)

# Extract text from file based on its extension
def extract_text_from_file(file_path):
    if file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    elif file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".doc"):
        return extract_text_from_doc(file_path)
    else:
        logging.error(f"Unsupported file format for {file_path}")
        return None

# Extract text from .docx files using mammoth and clean Unicode escapes
def extract_text_from_docx(file_path):
    try:
        with open(file_path, "rb") as docx_file:
            result = mammoth.extract_raw_text(docx_file)
            text = result.value
            return text.encode().decode('unicode_escape') if text else "No text found in the DOCX file"
    except Exception as e:
        logging.error(f"Error extracting text from .docx file {file_path}: {e}")
        return None

# Extract text from .pdf files using pdfplumber and clean Unicode escapes
def extract_text_from_pdf(file_path):
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text() or ""
                text += page_text  # Concatenate raw text
            return text.encode().decode('unicode_escape') if text else "No text found in the PDF file"
    except Exception as e:
        logging.error(f"Error extracting text from PDF file {file_path}: {e}")
        return None

# Extract text from .doc files using win32com
def extract_text_from_doc(file_path):
    try:
        word = win32.Dispatch("Word.Application")
        word.Visible = False
        doc = word.Documents.Open(file_path)
        text = doc.Content.Text
        doc.Close()
        word.Quit()
        return text.encode().decode("unicode_escape", "replace")
    except Exception as e:
        logging.error(f"Error extracting text from .doc file {file_path}: {e}")
        return None

# Function to replace characters in JSON files
def replace_characters_in_json_file(output_file):
    replacements = {
        "\u00e1": "á", "\u00e9": "é", "\u00ed": "í", "\u00f3": "ó", "\u00fa": "ú",
        "\u00c1": "Á", "\u00c9": "É", "\u00cd": "Í", "\u00d3": "Ó", "\u00da": "Ú",
        "\u00f1": "ñ", "\u00d1": "Ñ"
    }

    try:
        with open(output_file, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
        
        json_str = json.dumps(json_data)
        
        for unicode_seq, char in replacements.items():
            json_str = re.sub(unicode_seq, char, json_str)

        updated_json_data = json.loads(json_str)

        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(updated_json_data, file, ensure_ascii=False, indent=4)
        
        print("Character replacements in JSON file completed successfully.")
    
    except Exception as e:
        print(f"An error occurred while processing the JSON file: {e}")

# Example usage
file_path = "your_file_path_here"
extracted_text = extract_text_from_file(file_path)
print(extracted_text)


None


In [17]:

import win32com.client as win32
from docx import Document  # For extracting text from .docx
import mammoth
import pdfplumber
import os
import logging
import re

# Setup logging
logging.basicConfig(level=logging.INFO)

# Extract text from file based on its extension
def extract_text_from_file(file_path):
    if file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    elif file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".doc"):
        return extract_text_from_doc(file_path)
    else:
        logging.error(f"Unsupported file format for {file_path}")
        return None

# Extract text from .docx files using mammoth and clean Unicode escapes
def extract_text_from_docx(file_path):
    try:
        with open(file_path, "rb") as docx_file:
            result = mammoth.extract_raw_text(docx_file)
            text = result.value
            return text.encode().decode('unicode_escape') if text else "No text found in the DOCX file"
    except Exception as e:
        logging.error(f"Error extracting text from .docx file {file_path}: {e}")
        return None

# Extract text from .pdf files using pdfplumber and clean Unicode escapes
def extract_text_from_pdf(file_path):
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text() or ""
                text += page_text  # Concatenate raw text
            return text.encode().decode('unicode_escape') if text else "No text found in the PDF file"
    except Exception as e:
        logging.error(f"Error extracting text from PDF file {file_path}: {e}")
        return None

# Extract text from .doc files using win32com
def extract_text_from_doc(file_path):
    try:
        word = win32.Dispatch("Word.Application")
        word.Visible = False
        doc = word.Documents.Open(file_path)
        text = doc.Content.Text
        doc.Close()
        word.Quit()
        return text.encode().decode("unicode_escape", "replace")
    except Exception as e:
        logging.error(f"Error extracting text from .doc file {file_path}: {e}")
        return None

# Function to replace characters in JSON files
def replace_characters_in_json_file(output_file):
    replacements = {
        "\u00e1": "á", "\u00e9": "é", "\u00ed": "í", "\u00f3": "ó", "\u00fa": "ú",
        "\u00c1": "Á", "\u00c9": "É", "\u00cd": "Í", "\u00d3": "Ó", "\u00da": "Ú",
        "\u00f1": "ñ", "\u00d1": "Ñ"
    }

    try:
        with open(output_file, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
        
        json_str = json.dumps(json_data)
        
        for unicode_seq, char in replacements.items():
            json_str = re.sub(unicode_seq, char, json_str)

        updated_json_data = json.loads(json_str)

        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(updated_json_data, file, ensure_ascii=False, indent=4)
        
        print("Character replacements in JSON file completed successfully.")
    
    except Exception as e:
        print(f"An error occurred while processing the JSON file: {e}")

# Example usage
file_path = "your_file_path_here"
extracted_text = extract_text_from_file(file_path)
print(extracted_text)


None


In [18]:
# Extract the main domain (base URL) using the tld library
# Extraer el dominio principal (URL base) usando la biblioteca tld
# NO ESTAMOS USANDO ESTA FUNCION
# NO ES ACTIVADA  

def get_base_domain(url):
    try:
        # Extract the top-level domain (e.g., comunidadandina.org)
        return get_fld(url, fix_protocol=True)
    except Exception as e:
        print(f"Error extracting base domain from {url}: {e}")
        return urlparse(url).netloc

In [19]:

import win32com.client as win32
from docx import Document  # For extracting text from .docx
import mammoth
import pdfplumber
import os
import logging
import re

# Setup logging
logging.basicConfig(level=logging.INFO)

# Extract text from file based on its extension
def extract_text_from_file(file_path):
    if file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    elif file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".doc"):
        return extract_text_from_doc(file_path)
    else:
        logging.error(f"Unsupported file format for {file_path}")
        return None

# Extract text from .docx files using mammoth and clean Unicode escapes
def extract_text_from_docx(file_path):
    try:
        with open(file_path, "rb") as docx_file:
            result = mammoth.extract_raw_text(docx_file)
            text = result.value
            return text.encode().decode('unicode_escape') if text else "No text found in the DOCX file"
    except Exception as e:
        logging.error(f"Error extracting text from .docx file {file_path}: {e}")
        return None

# Extract text from .pdf files using pdfplumber and clean Unicode escapes
def extract_text_from_pdf(file_path):
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text() or ""
                text += page_text  # Concatenate raw text
            return text.encode().decode('unicode_escape') if text else "No text found in the PDF file"
    except Exception as e:
        logging.error(f"Error extracting text from PDF file {file_path}: {e}")
        return None

# Extract text from .doc files using win32com
def extract_text_from_doc(file_path):
    try:
        word = win32.Dispatch("Word.Application")
        word.Visible = False
        doc = word.Documents.Open(file_path)
        text = doc.Content.Text
        doc.Close()
        word.Quit()
        return text.encode().decode("unicode_escape", "replace")
    except Exception as e:
        logging.error(f"Error extracting text from .doc file {file_path}: {e}")
        return None

# Function to replace characters in JSON files
def replace_characters_in_json_file(output_file):
    replacements = {
        "\u00e1": "á", "\u00e9": "é", "\u00ed": "í", "\u00f3": "ó", "\u00fa": "ú",
        "\u00c1": "Á", "\u00c9": "É", "\u00cd": "Í", "\u00d3": "Ó", "\u00da": "Ú",
        "\u00f1": "ñ", "\u00d1": "Ñ"
    }

    try:
        with open(output_file, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
        
        json_str = json.dumps(json_data)
        
        for unicode_seq, char in replacements.items():
            json_str = re.sub(unicode_seq, char, json_str)

        updated_json_data = json.loads(json_str)

        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(updated_json_data, file, ensure_ascii=False, indent=4)
        
        print("Character replacements in JSON file completed successfully.")
    
    except Exception as e:
        print(f"An error occurred while processing the JSON file: {e}")

# Example usage
file_path = "your_file_path_here"
extracted_text = extract_text_from_file(file_path)
print(extracted_text)


None


In [20]:

import win32com.client as win32
from docx import Document  # For extracting text from .docx
import mammoth
import pdfplumber
import os
import logging
import re

# Setup logging
logging.basicConfig(level=logging.INFO)

# Extract text from file based on its extension
def extract_text_from_file(file_path):
    if file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    elif file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".doc"):
        return extract_text_from_doc(file_path)
    else:
        logging.error(f"Unsupported file format for {file_path}")
        return None

# Extract text from .docx files using mammoth and clean Unicode escapes
def extract_text_from_docx(file_path):
    try:
        with open(file_path, "rb") as docx_file:
            result = mammoth.extract_raw_text(docx_file)
            text = result.value
            return text.encode().decode('unicode_escape') if text else "No text found in the DOCX file"
    except Exception as e:
        logging.error(f"Error extracting text from .docx file {file_path}: {e}")
        return None

# Extract text from .pdf files using pdfplumber and clean Unicode escapes
def extract_text_from_pdf(file_path):
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text() or ""
                text += page_text  # Concatenate raw text
            return text.encode().decode('unicode_escape') if text else "No text found in the PDF file"
    except Exception as e:
        logging.error(f"Error extracting text from PDF file {file_path}: {e}")
        return None

# Extract text from .doc files using win32com
def extract_text_from_doc(file_path):
    try:
        word = win32.Dispatch("Word.Application")
        word.Visible = False
        doc = word.Documents.Open(file_path)
        text = doc.Content.Text
        doc.Close()
        word.Quit()
        return text.encode().decode("unicode_escape", "replace")
    except Exception as e:
        logging.error(f"Error extracting text from .doc file {file_path}: {e}")
        return None

# Function to replace characters in JSON files
def replace_characters_in_json_file(output_file):
    replacements = {
        "\u00e1": "á", "\u00e9": "é", "\u00ed": "í", "\u00f3": "ó", "\u00fa": "ú",
        "\u00c1": "Á", "\u00c9": "É", "\u00cd": "Í", "\u00d3": "Ó", "\u00da": "Ú",
        "\u00f1": "ñ", "\u00d1": "Ñ"
    }

    try:
        with open(output_file, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
        
        json_str = json.dumps(json_data)
        
        for unicode_seq, char in replacements.items():
            json_str = re.sub(unicode_seq, char, json_str)

        updated_json_data = json.loads(json_str)

        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(updated_json_data, file, ensure_ascii=False, indent=4)
        
        print("Character replacements in JSON file completed successfully.")
    
    except Exception as e:
        print(f"An error occurred while processing the JSON file: {e}")

# Example usage
file_path = "your_file_path_here"
extracted_text = extract_text_from_file(file_path)
print(extracted_text)


None


In [21]:
from tqdm import tqdm

# Assuming 'get_document_links' and 'to_camel_case' functions are defined

def scrape_and_store_links(urls):
    data = []  # List to hold the scraped links
    timestamp = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")

    for url in tqdm(urls, desc="Processing URLs"):  # Add tqdm progress bar to URLs loop
        print(f"Processing URL: {url}")
        base_url = url  # This is the parent page (where the link is found)

        doc_links = get_document_links(base_url, base_url, visited=set())  # Scrape the links from the base URL

        for parent_page, (link, doc_url) in tqdm(doc_links, desc="Extracting links", leave=False):  # Inner progress bar for each URL
            # Get the title from the link element (use link text)
            title = link.string or link.get_text().strip() or "No title"  # Fallback to "No title" if empty

            # Extract the path segments from the parent page (base_url)
            parsed_url = urlparse(parent_page)  # Use the parent_page returned by get_document_links
            path_segments = parsed_url.path.strip('/').split('/')

            # If path is empty, use domain (netloc) as the folder name
            if not path_segments or path_segments == ['']:
                path_segments = [parsed_url.netloc]  # Fallback to the domain if the path is empty
            else:
                # Convert to camelCase and skip empty segments
                path_segments = [to_camel_case(segment) for segment in path_segments if segment]

            # Append the details of each link to the data list, including the parent page path segments
            data.append({
                "timestamp": timestamp,
                "parent_page": path_segments,  # Store the path segments as an array
                "title": title,
                "href": link['href'],  # Extract href directly from the <a> tag (the document link)
                "file_url": doc_url,  # Full URL of the document
            })

    # Convert the list of links into a DataFrame and save as JSON
    df = pd.DataFrame(data)
    output_file = f"scraped_links_{timestamp}.json"
    df.to_json(output_file, orient="records", indent=4)
    print(f"Scraping complete. Links saved to {output_file}")
    return output_file


In [22]:

import win32com.client as win32
from docx import Document  # For extracting text from .docx
import mammoth
import pdfplumber
import os
import logging
import re

# Setup logging
logging.basicConfig(level=logging.INFO)

# Extract text from file based on its extension
def extract_text_from_file(file_path):
    if file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    elif file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".doc"):
        return extract_text_from_doc(file_path)
    else:
        logging.error(f"Unsupported file format for {file_path}")
        return None

# Extract text from .docx files using mammoth and clean Unicode escapes
def extract_text_from_docx(file_path):
    try:
        with open(file_path, "rb") as docx_file:
            result = mammoth.extract_raw_text(docx_file)
            text = result.value
            return text.encode().decode('unicode_escape') if text else "No text found in the DOCX file"
    except Exception as e:
        logging.error(f"Error extracting text from .docx file {file_path}: {e}")
        return None

# Extract text from .pdf files using pdfplumber and clean Unicode escapes
def extract_text_from_pdf(file_path):
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text() or ""
                text += page_text  # Concatenate raw text
            return text.encode().decode('unicode_escape') if text else "No text found in the PDF file"
    except Exception as e:
        logging.error(f"Error extracting text from PDF file {file_path}: {e}")
        return None

# Extract text from .doc files using win32com
def extract_text_from_doc(file_path):
    try:
        word = win32.Dispatch("Word.Application")
        word.Visible = False
        doc = word.Documents.Open(file_path)
        text = doc.Content.Text
        doc.Close()
        word.Quit()
        return text.encode().decode("unicode_escape", "replace")
    except Exception as e:
        logging.error(f"Error extracting text from .doc file {file_path}: {e}")
        return None

# Function to replace characters in JSON files
def replace_characters_in_json_file(output_file):
    replacements = {
        "\u00e1": "á", "\u00e9": "é", "\u00ed": "í", "\u00f3": "ó", "\u00fa": "ú",
        "\u00c1": "Á", "\u00c9": "É", "\u00cd": "Í", "\u00d3": "Ó", "\u00da": "Ú",
        "\u00f1": "ñ", "\u00d1": "Ñ"
    }

    try:
        with open(output_file, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
        
        json_str = json.dumps(json_data)
        
        for unicode_seq, char in replacements.items():
            json_str = re.sub(unicode_seq, char, json_str)

        updated_json_data = json.loads(json_str)

        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(updated_json_data, file, ensure_ascii=False, indent=4)
        
        print("Character replacements in JSON file completed successfully.")
    
    except Exception as e:
        print(f"An error occurred while processing the JSON file: {e}")

# Example usage
file_path = "your_file_path_here"
extracted_text = extract_text_from_file(file_path)
print(extracted_text)


None


In [23]:

import win32com.client as win32
from docx import Document  # For extracting text from .docx
import mammoth
import pdfplumber
import os
import logging
import re

# Setup logging
logging.basicConfig(level=logging.INFO)

# Extract text from file based on its extension
def extract_text_from_file(file_path):
    if file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    elif file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".doc"):
        return extract_text_from_doc(file_path)
    else:
        logging.error(f"Unsupported file format for {file_path}")
        return None

# Extract text from .docx files using mammoth and clean Unicode escapes
def extract_text_from_docx(file_path):
    try:
        with open(file_path, "rb") as docx_file:
            result = mammoth.extract_raw_text(docx_file)
            text = result.value
            return text.encode().decode('unicode_escape') if text else "No text found in the DOCX file"
    except Exception as e:
        logging.error(f"Error extracting text from .docx file {file_path}: {e}")
        return None

# Extract text from .pdf files using pdfplumber and clean Unicode escapes
def extract_text_from_pdf(file_path):
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text() or ""
                text += page_text  # Concatenate raw text
            return text.encode().decode('unicode_escape') if text else "No text found in the PDF file"
    except Exception as e:
        logging.error(f"Error extracting text from PDF file {file_path}: {e}")
        return None

# Extract text from .doc files using win32com
def extract_text_from_doc(file_path):
    try:
        word = win32.Dispatch("Word.Application")
        word.Visible = False
        doc = word.Documents.Open(file_path)
        text = doc.Content.Text
        doc.Close()
        word.Quit()
        return text.encode().decode("unicode_escape", "replace")
    except Exception as e:
        logging.error(f"Error extracting text from .doc file {file_path}: {e}")
        return None

# Function to replace characters in JSON files
def replace_characters_in_json_file(output_file):
    replacements = {
        "\u00e1": "á", "\u00e9": "é", "\u00ed": "í", "\u00f3": "ó", "\u00fa": "ú",
        "\u00c1": "Á", "\u00c9": "É", "\u00cd": "Í", "\u00d3": "Ó", "\u00da": "Ú",
        "\u00f1": "ñ", "\u00d1": "Ñ"
    }

    try:
        with open(output_file, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
        
        json_str = json.dumps(json_data)
        
        for unicode_seq, char in replacements.items():
            json_str = re.sub(unicode_seq, char, json_str)

        updated_json_data = json.loads(json_str)

        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(updated_json_data, file, ensure_ascii=False, indent=4)
        
        print("Character replacements in JSON file completed successfully.")
    
    except Exception as e:
        print(f"An error occurred while processing the JSON file: {e}")

# Example usage
file_path = "your_file_path_here"
extracted_text = extract_text_from_file(file_path)
print(extracted_text)


None


In [24]:

import win32com.client as win32
from docx import Document  # For extracting text from .docx
import mammoth
import pdfplumber
import os
import logging
import re

# Setup logging
logging.basicConfig(level=logging.INFO)

# Extract text from file based on its extension
def extract_text_from_file(file_path):
    if file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    elif file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".doc"):
        return extract_text_from_doc(file_path)
    else:
        logging.error(f"Unsupported file format for {file_path}")
        return None

# Extract text from .docx files using mammoth and clean Unicode escapes
def extract_text_from_docx(file_path):
    try:
        with open(file_path, "rb") as docx_file:
            result = mammoth.extract_raw_text(docx_file)
            text = result.value
            return text.encode().decode('unicode_escape') if text else "No text found in the DOCX file"
    except Exception as e:
        logging.error(f"Error extracting text from .docx file {file_path}: {e}")
        return None

# Extract text from .pdf files using pdfplumber and clean Unicode escapes
def extract_text_from_pdf(file_path):
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text() or ""
                text += page_text  # Concatenate raw text
            return text.encode().decode('unicode_escape') if text else "No text found in the PDF file"
    except Exception as e:
        logging.error(f"Error extracting text from PDF file {file_path}: {e}")
        return None

# Extract text from .doc files using win32com
def extract_text_from_doc(file_path):
    try:
        word = win32.Dispatch("Word.Application")
        word.Visible = False
        doc = word.Documents.Open(file_path)
        text = doc.Content.Text
        doc.Close()
        word.Quit()
        return text.encode().decode("unicode_escape", "replace")
    except Exception as e:
        logging.error(f"Error extracting text from .doc file {file_path}: {e}")
        return None

# Function to replace characters in JSON files
def replace_characters_in_json_file(output_file):
    replacements = {
        "\u00e1": "á", "\u00e9": "é", "\u00ed": "í", "\u00f3": "ó", "\u00fa": "ú",
        "\u00c1": "Á", "\u00c9": "É", "\u00cd": "Í", "\u00d3": "Ó", "\u00da": "Ú",
        "\u00f1": "ñ", "\u00d1": "Ñ"
    }

    try:
        with open(output_file, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
        
        json_str = json.dumps(json_data)
        
        for unicode_seq, char in replacements.items():
            json_str = re.sub(unicode_seq, char, json_str)

        updated_json_data = json.loads(json_str)

        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(updated_json_data, file, ensure_ascii=False, indent=4)
        
        print("Character replacements in JSON file completed successfully.")
    
    except Exception as e:
        print(f"An error occurred while processing the JSON file: {e}")

# Example usage
file_path = "your_file_path_here"
extracted_text = extract_text_from_file(file_path)
print(extracted_text)


None


In [25]:

import win32com.client as win32
from docx import Document  # For extracting text from .docx
import mammoth
import pdfplumber
import os
import logging
import re

# Setup logging
logging.basicConfig(level=logging.INFO)

# Extract text from file based on its extension
def extract_text_from_file(file_path):
    if file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    elif file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".doc"):
        return extract_text_from_doc(file_path)
    else:
        logging.error(f"Unsupported file format for {file_path}")
        return None

# Extract text from .docx files using mammoth and clean Unicode escapes
def extract_text_from_docx(file_path):
    try:
        with open(file_path, "rb") as docx_file:
            result = mammoth.extract_raw_text(docx_file)
            text = result.value
            return text.encode().decode('unicode_escape') if text else "No text found in the DOCX file"
    except Exception as e:
        logging.error(f"Error extracting text from .docx file {file_path}: {e}")
        return None

# Extract text from .pdf files using pdfplumber and clean Unicode escapes
def extract_text_from_pdf(file_path):
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text() or ""
                text += page_text  # Concatenate raw text
            return text.encode().decode('unicode_escape') if text else "No text found in the PDF file"
    except Exception as e:
        logging.error(f"Error extracting text from PDF file {file_path}: {e}")
        return None

# Extract text from .doc files using win32com
def extract_text_from_doc(file_path):
    try:
        word = win32.Dispatch("Word.Application")
        word.Visible = False
        doc = word.Documents.Open(file_path)
        text = doc.Content.Text
        doc.Close()
        word.Quit()
        return text.encode().decode("unicode_escape", "replace")
    except Exception as e:
        logging.error(f"Error extracting text from .doc file {file_path}: {e}")
        return None

# Function to replace characters in JSON files
def replace_characters_in_json_file(output_file):
    replacements = {
        "\u00e1": "á", "\u00e9": "é", "\u00ed": "í", "\u00f3": "ó", "\u00fa": "ú",
        "\u00c1": "Á", "\u00c9": "É", "\u00cd": "Í", "\u00d3": "Ó", "\u00da": "Ú",
        "\u00f1": "ñ", "\u00d1": "Ñ"
    }

    try:
        with open(output_file, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
        
        json_str = json.dumps(json_data)
        
        for unicode_seq, char in replacements.items():
            json_str = re.sub(unicode_seq, char, json_str)

        updated_json_data = json.loads(json_str)

        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(updated_json_data, file, ensure_ascii=False, indent=4)
        
        print("Character replacements in JSON file completed successfully.")
    
    except Exception as e:
        print(f"An error occurred while processing the JSON file: {e}")

# Example usage
file_path = "your_file_path_here"
extracted_text = extract_text_from_file(file_path)
print(extracted_text)


None


In [26]:

import win32com.client as win32
from docx import Document  # For extracting text from .docx
import mammoth
import pdfplumber
import os
import logging
import re

# Setup logging
logging.basicConfig(level=logging.INFO)

# Extract text from file based on its extension
def extract_text_from_file(file_path):
    if file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    elif file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".doc"):
        return extract_text_from_doc(file_path)
    else:
        logging.error(f"Unsupported file format for {file_path}")
        return None

# Extract text from .docx files using mammoth and clean Unicode escapes
def extract_text_from_docx(file_path):
    try:
        with open(file_path, "rb") as docx_file:
            result = mammoth.extract_raw_text(docx_file)
            text = result.value
            return text.encode().decode('unicode_escape') if text else "No text found in the DOCX file"
    except Exception as e:
        logging.error(f"Error extracting text from .docx file {file_path}: {e}")
        return None

# Extract text from .pdf files using pdfplumber and clean Unicode escapes
def extract_text_from_pdf(file_path):
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text() or ""
                text += page_text  # Concatenate raw text
            return text.encode().decode('unicode_escape') if text else "No text found in the PDF file"
    except Exception as e:
        logging.error(f"Error extracting text from PDF file {file_path}: {e}")
        return None

# Extract text from .doc files using win32com
def extract_text_from_doc(file_path):
    try:
        word = win32.Dispatch("Word.Application")
        word.Visible = False
        doc = word.Documents.Open(file_path)
        text = doc.Content.Text
        doc.Close()
        word.Quit()
        return text.encode().decode("unicode_escape", "replace")
    except Exception as e:
        logging.error(f"Error extracting text from .doc file {file_path}: {e}")
        return None

# Function to replace characters in JSON files
def replace_characters_in_json_file(output_file):
    replacements = {
        "\u00e1": "á", "\u00e9": "é", "\u00ed": "í", "\u00f3": "ó", "\u00fa": "ú",
        "\u00c1": "Á", "\u00c9": "É", "\u00cd": "Í", "\u00d3": "Ó", "\u00da": "Ú",
        "\u00f1": "ñ", "\u00d1": "Ñ"
    }

    try:
        with open(output_file, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
        
        json_str = json.dumps(json_data)
        
        for unicode_seq, char in replacements.items():
            json_str = re.sub(unicode_seq, char, json_str)

        updated_json_data = json.loads(json_str)

        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(updated_json_data, file, ensure_ascii=False, indent=4)
        
        print("Character replacements in JSON file completed successfully.")
    
    except Exception as e:
        print(f"An error occurred while processing the JSON file: {e}")

# Example usage
file_path = "your_file_path_here"
extracted_text = extract_text_from_file(file_path)
print(extracted_text)


None


In [27]:

import win32com.client as win32
from docx import Document  # For extracting text from .docx
import mammoth
import pdfplumber
import os
import logging
import re

# Setup logging
logging.basicConfig(level=logging.INFO)

# Extract text from file based on its extension
def extract_text_from_file(file_path):
    if file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    elif file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".doc"):
        return extract_text_from_doc(file_path)
    else:
        logging.error(f"Unsupported file format for {file_path}")
        return None

# Extract text from .docx files using mammoth and clean Unicode escapes
def extract_text_from_docx(file_path):
    try:
        with open(file_path, "rb") as docx_file:
            result = mammoth.extract_raw_text(docx_file)
            text = result.value
            return text.encode().decode('unicode_escape') if text else "No text found in the DOCX file"
    except Exception as e:
        logging.error(f"Error extracting text from .docx file {file_path}: {e}")
        return None

# Extract text from .pdf files using pdfplumber and clean Unicode escapes
def extract_text_from_pdf(file_path):
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text() or ""
                text += page_text  # Concatenate raw text
            return text.encode().decode('unicode_escape') if text else "No text found in the PDF file"
    except Exception as e:
        logging.error(f"Error extracting text from PDF file {file_path}: {e}")
        return None

# Extract text from .doc files using win32com
def extract_text_from_doc(file_path):
    try:
        word = win32.Dispatch("Word.Application")
        word.Visible = False
        doc = word.Documents.Open(file_path)
        text = doc.Content.Text
        doc.Close()
        word.Quit()
        return text.encode().decode("unicode_escape", "replace")
    except Exception as e:
        logging.error(f"Error extracting text from .doc file {file_path}: {e}")
        return None

# Function to replace characters in JSON files
def replace_characters_in_json_file(output_file):
    replacements = {
        "\u00e1": "á", "\u00e9": "é", "\u00ed": "í", "\u00f3": "ó", "\u00fa": "ú",
        "\u00c1": "Á", "\u00c9": "É", "\u00cd": "Í", "\u00d3": "Ó", "\u00da": "Ú",
        "\u00f1": "ñ", "\u00d1": "Ñ"
    }

    try:
        with open(output_file, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
        
        json_str = json.dumps(json_data)
        
        for unicode_seq, char in replacements.items():
            json_str = re.sub(unicode_seq, char, json_str)

        updated_json_data = json.loads(json_str)

        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(updated_json_data, file, ensure_ascii=False, indent=4)
        
        print("Character replacements in JSON file completed successfully.")
    
    except Exception as e:
        print(f"An error occurred while processing the JSON file: {e}")

# Example usage
file_path = "your_file_path_here"
extracted_text = extract_text_from_file(file_path)
print(extracted_text)


None


In [28]:

import win32com.client as win32
from docx import Document  # For extracting text from .docx
import mammoth
import pdfplumber
import os
import logging
import re

# Setup logging
logging.basicConfig(level=logging.INFO)

# Extract text from file based on its extension
def extract_text_from_file(file_path):
    if file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    elif file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".doc"):
        return extract_text_from_doc(file_path)
    else:
        logging.error(f"Unsupported file format for {file_path}")
        return None

# Extract text from .docx files using mammoth and clean Unicode escapes
def extract_text_from_docx(file_path):
    try:
        with open(file_path, "rb") as docx_file:
            result = mammoth.extract_raw_text(docx_file)
            text = result.value
            return text.encode().decode('unicode_escape') if text else "No text found in the DOCX file"
    except Exception as e:
        logging.error(f"Error extracting text from .docx file {file_path}: {e}")
        return None

# Extract text from .pdf files using pdfplumber and clean Unicode escapes
def extract_text_from_pdf(file_path):
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text() or ""
                text += page_text  # Concatenate raw text
            return text.encode().decode('unicode_escape') if text else "No text found in the PDF file"
    except Exception as e:
        logging.error(f"Error extracting text from PDF file {file_path}: {e}")
        return None

# Extract text from .doc files using win32com
def extract_text_from_doc(file_path):
    try:
        word = win32.Dispatch("Word.Application")
        word.Visible = False
        doc = word.Documents.Open(file_path)
        text = doc.Content.Text
        doc.Close()
        word.Quit()
        return text.encode().decode("unicode_escape", "replace")
    except Exception as e:
        logging.error(f"Error extracting text from .doc file {file_path}: {e}")
        return None

# Function to replace characters in JSON files
def replace_characters_in_json_file(output_file):
    replacements = {
        "\u00e1": "á", "\u00e9": "é", "\u00ed": "í", "\u00f3": "ó", "\u00fa": "ú",
        "\u00c1": "Á", "\u00c9": "É", "\u00cd": "Í", "\u00d3": "Ó", "\u00da": "Ú",
        "\u00f1": "ñ", "\u00d1": "Ñ"
    }

    try:
        with open(output_file, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
        
        json_str = json.dumps(json_data)
        
        for unicode_seq, char in replacements.items():
            json_str = re.sub(unicode_seq, char, json_str)

        updated_json_data = json.loads(json_str)

        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(updated_json_data, file, ensure_ascii=False, indent=4)
        
        print("Character replacements in JSON file completed successfully.")
    
    except Exception as e:
        print(f"An error occurred while processing the JSON file: {e}")

# Example usage
file_path = "your_file_path_here"
extracted_text = extract_text_from_file(file_path)
print(extracted_text)


None


In [29]:

import win32com.client as win32
from docx import Document  # For extracting text from .docx
import mammoth
import pdfplumber
import os
import logging
import re

# Setup logging
logging.basicConfig(level=logging.INFO)

# Extract text from file based on its extension
def extract_text_from_file(file_path):
    if file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    elif file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".doc"):
        return extract_text_from_doc(file_path)
    else:
        logging.error(f"Unsupported file format for {file_path}")
        return None

# Extract text from .docx files using mammoth and clean Unicode escapes
def extract_text_from_docx(file_path):
    try:
        with open(file_path, "rb") as docx_file:
            result = mammoth.extract_raw_text(docx_file)
            text = result.value
            return text.encode().decode('unicode_escape') if text else "No text found in the DOCX file"
    except Exception as e:
        logging.error(f"Error extracting text from .docx file {file_path}: {e}")
        return None

# Extract text from .pdf files using pdfplumber and clean Unicode escapes
def extract_text_from_pdf(file_path):
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text() or ""
                text += page_text  # Concatenate raw text
            return text.encode().decode('unicode_escape') if text else "No text found in the PDF file"
    except Exception as e:
        logging.error(f"Error extracting text from PDF file {file_path}: {e}")
        return None

# Extract text from .doc files using win32com
def extract_text_from_doc(file_path):
    try:
        word = win32.Dispatch("Word.Application")
        word.Visible = False
        doc = word.Documents.Open(file_path)
        text = doc.Content.Text
        doc.Close()
        word.Quit()
        return text.encode().decode("unicode_escape", "replace")
    except Exception as e:
        logging.error(f"Error extracting text from .doc file {file_path}: {e}")
        return None

# Function to replace characters in JSON files
def replace_characters_in_json_file(output_file):
    replacements = {
        "\u00e1": "á", "\u00e9": "é", "\u00ed": "í", "\u00f3": "ó", "\u00fa": "ú",
        "\u00c1": "Á", "\u00c9": "É", "\u00cd": "Í", "\u00d3": "Ó", "\u00da": "Ú",
        "\u00f1": "ñ", "\u00d1": "Ñ"
    }

    try:
        with open(output_file, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
        
        json_str = json.dumps(json_data)
        
        for unicode_seq, char in replacements.items():
            json_str = re.sub(unicode_seq, char, json_str)

        updated_json_data = json.loads(json_str)

        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(updated_json_data, file, ensure_ascii=False, indent=4)
        
        print("Character replacements in JSON file completed successfully.")
    
    except Exception as e:
        print(f"An error occurred while processing the JSON file: {e}")

# Example usage
file_path = "your_file_path_here"
extracted_text = extract_text_from_file(file_path)
print(extracted_text)


None


In [30]:

import win32com.client as win32
from docx import Document  # For extracting text from .docx
import mammoth
import pdfplumber
import os
import logging
import re

# Setup logging
logging.basicConfig(level=logging.INFO)

# Extract text from file based on its extension
def extract_text_from_file(file_path):
    if file_path.endswith(".docx"):
        return extract_text_from_docx(file_path)
    elif file_path.endswith(".pdf"):
        return extract_text_from_pdf(file_path)
    elif file_path.endswith(".doc"):
        return extract_text_from_doc(file_path)
    else:
        logging.error(f"Unsupported file format for {file_path}")
        return None

# Extract text from .docx files using mammoth and clean Unicode escapes
def extract_text_from_docx(file_path):
    try:
        with open(file_path, "rb") as docx_file:
            result = mammoth.extract_raw_text(docx_file)
            text = result.value
            return text.encode().decode('unicode_escape') if text else "No text found in the DOCX file"
    except Exception as e:
        logging.error(f"Error extracting text from .docx file {file_path}: {e}")
        return None

# Extract text from .pdf files using pdfplumber and clean Unicode escapes
def extract_text_from_pdf(file_path):
    try:
        with pdfplumber.open(file_path) as pdf:
            text = ""
            for page in pdf.pages:
                page_text = page.extract_text() or ""
                text += page_text  # Concatenate raw text
            return text.encode().decode('unicode_escape') if text else "No text found in the PDF file"
    except Exception as e:
        logging.error(f"Error extracting text from PDF file {file_path}: {e}")
        return None

# Extract text from .doc files using win32com
def extract_text_from_doc(file_path):
    try:
        word = win32.Dispatch("Word.Application")
        word.Visible = False
        doc = word.Documents.Open(file_path)
        text = doc.Content.Text
        doc.Close()
        word.Quit()
        return text.encode().decode("unicode_escape", "replace")
    except Exception as e:
        logging.error(f"Error extracting text from .doc file {file_path}: {e}")
        return None

# Function to replace characters in JSON files
def replace_characters_in_json_file(output_file):
    replacements = {
        "\u00e1": "á", "\u00e9": "é", "\u00ed": "í", "\u00f3": "ó", "\u00fa": "ú",
        "\u00c1": "Á", "\u00c9": "É", "\u00cd": "Í", "\u00d3": "Ó", "\u00da": "Ú",
        "\u00f1": "ñ", "\u00d1": "Ñ"
    }

    try:
        with open(output_file, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
        
        json_str = json.dumps(json_data)
        
        for unicode_seq, char in replacements.items():
            json_str = re.sub(unicode_seq, char, json_str)

        updated_json_data = json.loads(json_str)

        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(updated_json_data, file, ensure_ascii=False, indent=4)
        
        print("Character replacements in JSON file completed successfully.")
    
    except Exception as e:
        print(f"An error occurred while processing the JSON file: {e}")

# Example usage
file_path = "your_file_path_here"
extracted_text = extract_text_from_file(file_path)
print(extracted_text)


None


In [31]:
# scrape_and_store_link

urls = read_urls()
scrape_and_store_links(urls)

Processing URLs:   0%|          | 0/1 [00:00<?, ?it/s]

Processing URL: https://www.comunidadandina.org/
Processing: tel:+5117106573unidadandina.org/archivo-de-convocatorias/e-la-secretaria-general-can//a/
Error fetching tel:+5117106573: No connection adapters were found for 'tel:+5117106573'
Processing: mailto:bibliocan@comunidadandina.org
Error fetching mailto:bibliocan@comunidadandina.org: No connection adapters were found for 'mailto:bibliocan@comunidadandina.org'


Processing URLs:   0%|          | 0/1 [00:26<?, ?it/s]


KeyboardInterrupt: 

In [33]:
download_documents_from_json('scraped_links_2024-10-15_20-26-53.json')

NameError: name 'download_documents_from_json' is not defined

In [45]:
import re
import json

def replace_characters_in_json_file(output_file):
    # Define the replacements for common problematic characters
    replacements = {
        r'\u00e1': 'á', r'\u00e9': 'é', r'\u00ed': 'í', r'\u00f3': 'ó', r'\u00fa': 'ú',
        r'\u00f1': 'ñ', r'\u00c1': 'Á', r'\u00c9': 'É', r'\u00cd': 'Í', r'\u00d3': 'Ó',
        r'\u00da': 'Ú', r'\u00d1': 'Ñ'
    }
    
    try:
        # Step 1: Read the JSON file
        with open(output_file, 'r', encoding='utf-8') as file:
            json_data = json.load(file)
        
        # Step 2: Convert JSON to string and perform replacements
        json_str = json.dumps(json_data)  # Convert JSON data to string
        
        # Perform replacements using the dictionary
        for unicode_seq, char in replacements.items():
            json_str = re.sub(unicode_seq, char, json_str)
        
        # Step 3: Convert the string back to JSON format
        updated_json_data = json.loads(json_str)
        
        # Step 4: Write the updated JSON back to the file
        with open(output_file, 'w', encoding='utf-8') as file:
            json.dump(updated_json_data, file, ensure_ascii=False, indent=4)
        
        print("Character replacements in JSON file completed successfully.")
    
    except Exception as e:
        print(f"An error occurred while processing the JSON file: {e}")

# Usage
replace_characters_in_json_file('results_with_download_2024-10-17T18-19-41.369908.json')


Character replacements in JSON file completed successfully.


In [ ]:
# Función principal que organiza el flujo de scraping, almacenamiento y descarga de documentos
def main():
    urls = read_urls()  # Read URLs from the text file

    # Phase 1: Scrape and store links in JSON
    links_json_file = scrape_and_store_links(urls)
    #links_json_file = "scraped_links_2024-10-09.json"

    # Phase 2: Download documents using the stored links
    download_documents_from_json(links_json_file)


if __name__ == "__main__":
    main()
